# 数据清理

## URL 合并

In [28]:
# 导入数据文件

# here put the import lib
import os

import logging as log
import pandas as pd

from modules.logger.logger import init as logInit
from modules.logUtils.httpEvent import HttpEvent, TABLE_LABELS
from modules.fileUtils.fileUtils import getFilesByPath
from modules.config.argumentParser import argumentParser
from modules.config.configParser import getValue as getValueFromConfig

# CONFIG

TABLE_LABELS = [
    "timestamp",
    "threadId",
    "institutionId",
    "userId",
    "url",
    "method",
    "statusCode",
    "parameterType",
    "parameterName",
    "parameterValue",
    "header",
    "name",
    "ip",
    "port",
    "vpnip"
]
# get all files.
files = getFilesByPath("data/origin")
if not files: 
    print("there no any file. recorrect the input path.")
files = files

import random

# 随机抽取 RANDOM_FILES_NUMBER 数量的日志
RANDOM_FILES_NUMBER = 0
files = random.sample(files, RANDOM_FILES_NUMBER) if RANDOM_FILES_NUMBER != 0 else files

print(len(files))
# print(files)

994


In [29]:
# 将数据读取到 DataFrame 中

data = []
for file in files:
    count = 0
    with open(file, 'r', encoding='utf-8') as f:
        # read raw log by line.
        for line in f:
            try:
                he = HttpEvent(line)
                
                # filter user
                if he.userId == 'null': continue

                # append to Data
                data.extend([he.getSet()])
                count += 1
            except Exception as e:
                log.debug(e)
    log.info("{0} write {1} lines".format(file, count))
print(len(data), len(data[0]))
# convert List to DataFrame
# print(data[0])
df = pd.DataFrame(data = data, columns=TABLE_LABELS)
del data

2424864 15


In [30]:
# 保存到 HDF
df.to_csv("data/tmp/full.csv", sep="^")
!ls -l data/tmp/full.csv

-rw------- 1 chase chase 1441753933 12月  9 11:39 data/tmp/full.csv


In [ ]:
# 从 CSV 读取
import pandas as pd
df = pd.read_csv('data/tmp/full.csv', sep='^')

In [ ]:
# 查看机构号
institutesDF = df.institutionId
institutes = institutesDF.value_counts()
for k, v in institutes.items():
    print(v, k)

In [ ]:
# 查看 userId
userIdDF = df.userId
users = userIdDF.value_counts()
for k, v in users.items():
    print(v, k)
print(len(users))

In [ ]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

# for k, v in urls.items():
#     print(v, k)

print(len(urls))

In [34]:
# convert all urls
from modules.logUtils.urlFilter import urlConvert
df['url'] = df['url'].apply(urlConvert, args=(set(df.userId.values), set(df.institutionId.values),  set(df.name.values),))
# save to CSV
df.to_csv("data/tmp/converedtUrl_full.csv", sep='^')

In [68]:
# 输出所有 url 和次数
urlDF = df.url
urls = urlDF.value_counts()

# for k, v in urls.items():
#     print(v, k)

print(len(urls))

1633


In [80]:
# delete bug log
df = df.drop(index=(df.loc[(df.name=='何群辉')].index))
# delete unused log
df = df.drop(index=(df.loc[(df.header=="null")].index))
# delete unused column
df = df.drop(['threadId', 'statusCode', 'parameterType'], axis=1)

## session 提取

In [81]:
# Sort by userId primary key, with time as the secondary key
df.sort_values(by=['userId', 'timestamp'])

,timestamp,institutionId,userId,url,method,parameterName,parameterValue,header,name,ip,port,vpnip
99068,1598497193.69,310111,053982f7c98843c0b3bccb1b06723789,/portal/shortcutconfig/getConfigCompletion,GET,[],[],"[{""httpOnly"":false,""maxAge"":-1,""name"":""JSESSIO...",秦颖,159.226.195.234,54040,0
99069,1598497193.691,310111,053982f7c98843c0b3bccb1b06723789,/portal/menus/mine/pc,GET,"[""image""]",[false],"[{""httpOnly"":false,""maxAge"":-1,""name"":""JSESSIO...",秦颖,159.226.195.234,54041,0
99070,1598497193.744,310111,053982f7c98843c0b3bccb1b06723789,/portal/menus,POST,"[""ids"",""image""]","[[""c7ee14851f954a5096c1538fc291b670"",""0e8049b3...","[{""httpOnly"":false,""maxAge"":-1,""name"":""JSESSIO...",秦颖,159.226.195.234,54039,0
99071,1598497193.84,310111,053982f7c98843c0b3bccb1b06723789,/portal/shortcutconfig/getConfigCompletion,GET,[],[],"[{""httpOnly"":false,""maxAge"":-1,""name"":""JSESSIO...",秦颖,159.226.195.234,54043,0
619274,1598497193.875,310111,053982f7c98843c0b3bccb1b06723789,/hr/thrpeople/,POST,[],[],"[{""httpOnly"":false,""maxAge"":-1,""name"":""Authori...",秦颖,159.226.195.234,54043,0
...,...,...,...,...,...,...,...,...,...,...,...,...
170503,1598842670.014,310111,f7039e03ff224c9293ef82381e5849e7,/od/tOdQpjOrgids/querySignProcess,POST,"[""params""]","[{""ngrid"":""f7039e03ff224c9293ef82381e5849e7"",""...","[{""httpOnly"":false,""maxAge"":-1,""name"":""Authori...",曹天贵,159.226.99.33,52510,0
170504,1598842670.046,310111,f7039e03ff224c9293ef82381e5849e7,/od/tOdArchivesInOrgids/queryInProcess,POST,"[""params""]","[{""ngrid"":""f7039e03ff224c9293ef82381e5849e7"",""...","[{""httpOnly"":false,""maxAge"":-1,""name"":""Authori...",曹天贵,159.226.99.33,52512,0
631229,1598842670.513,310111,f7039e03ff224c9293ef82381e5849e7,/fin/arBill/getArBillAllTODOCount,GET,[],[],"[{""httpOnly"":false,""maxAge"":-1,""name"":""Authori...",曹天贵,159.226.99.33,52504,0
123806,1598842670.676,310111,f7039e03ff224c9293ef82381e5849e7,/pm/commoncore/todoCommonCore/getPmTotalPendin...,GET,"[""isMobile""]",[null],"[{""httpOnly"":false,""maxAge"":-1,""name"":""Authori...",曹天贵,159.226.99.33,52504,0


杨帆     92864
王芳     89032
高辉     44234
简乐     26318
文印室    24281
       ...  
胡泊        17
陈枢舒       16
秦甜        15
陈灿        12
贾萌         9
Name: name, Length: 847, dtype: int64